In [14]:
import numpy as np
import pandas as pd
import requests
from pykrx import stock
import time
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

name = '펄어비스'
fromdate = str(20210101)
todate = str(20211104)

ticker1 = stock_list.loc[stock_list['종목명']== name, '종목코드']
df1 = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker1)

seri = df1['종가']
df = pd.DataFrame(seri, columns=['종가','일계도','이계도','일계도부호','이계도부호'])

df['일계도'][0] = 0
for i in range(1, len(seri)):
    df['일계도'][i] = df['종가'][i] - df['종가'][i-1]

df['이계도'][0] = 0
for i in range(1, len(seri)):
    df['이계도'][i] = df['일계도'][i] - df['일계도'][i-1]

for i in range(len(df)):
    if df['일계도'][i] >= 0:
        df['일계도부호'][i] = 1
    else:
        df['일계도부호'][i] = 0

for i in range(len(df)):
    if df['이계도'][i] >= 0:
        df['이계도부호'][i] = 1
    else:
        df['이계도부호'][i] = 0

df_diff2 = df['일계도부호']
df_diff2 = np.array(df_diff2)

C:\Users\ghdak\AppData\Local\Temp/ipykernel_19916/3790775526.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일계도'][0] = 0
C:\Users\ghdak\AppData\Local\Temp/ipykernel_19916/3790775526.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일계도'][i] = df['종가'][i] - df['종가'][i-1]
C:\Users\ghdak\AppData\Local\Temp/ipykernel_19916/3790775526.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['이계도'][0] = 0
C:\Users\ghdak\AppDa

In [20]:
test_x = [df_diff2[i:i+40] for i in range(len(df_diff2[:150]))]
test_y = [df_diff2[i+41] for i in range(len(df_diff2[:150]))]

In [23]:
import import_ipynb
from momentum import model

In [24]:
pred_y = model.predict(test_x)
pred_y

C:\Users\ghdak\anaconda3\envs\gpu_env\lib\site-packages\sklearn\base.py:561: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


ValueError: X has 40 features, but LinearRegression is expecting 30 features as input.

In [18]:
trans_pred = pred_y.copy()
for i in range(len(pred_y)):
    if pred_y[i] < 0.5:
        trans_pred[i] = 0
    else:
        trans_pred[i] = 1

In [19]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_y, trans_pred)
round(mae, 2)

0.42